----
# Auto-ARIMA Forecasting
-----

In this notebook, I explore timeseries forecasting of stock returns using : ARIMA, SARIMA, and SARIMAX. For this analysis, I will be using the pmdarima package. This library simplifies the implementation and tuning of ARIMA and its paramters - this is particularly helpful when forecasting stock related data due to the high volatility. 

### Notebook Overview

The purpose of this project is to predict the close price of Microsfot Stock for the next 7 days. To do this, I will:

1. **Data Loading and Preparation:** Load stock data

2. **Forecasting with Auto ARIMA:** Use pmdarima to automatically identify the best paramters for ARIMA forecasting.

3. **Incorporate Seasonality with SARIMA:** Extend the ARIMA model to capture seasonal effects using pmdarima.

4. **Further Enhance Forecasting with SARIMAX:** Use exogenous variables such as interest rates to improve forecasting accuracy.

5. **Evaluate Model Performance:** Assess the accuracy of our forecasts and analyse residuals to ensure the model’s effectiveness.


## Set Up
----

In [2]:
import numpy as np
import pandas as pd

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.api import tsa # time series analysis
import statsmodels.api as sm
import pmdarima as pm
from statsmodels.tsa.stattools import adfuller
from scipy.stats import boxcox
from scipy.special import inv_boxcox
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit

## Utility Functions
----

In [3]:
def adf_test(series, threshold = 0.05):
    """
    Description:
    Perform the Augmented Dickey-Fuller (ADF) test to check for stationarity.

    Parameters:
    - series : The time series data to test for stationarity.
    - threshold : The significance level to determine stationarity (default is 0.05).

    Returns:
    - None: Prints the ADF statistic, p-value and stationarity result.
    """
    adf_test = adfuller(series)
    adf_statistic = adf_test[0]
    p_value = adf_test[1]
    
    print(f'ADF Statistic: {adf_statistic}')
    print(f'p-value: {p_value}')
 
    if p_value < threshold:
        print("Data is stationary.")
    else:
        print("Data is not stationary.")

In [4]:
def plt_pred_vs_actual(actual, predicted):
    
    """
    Description:
    Plots the actual versus forecasted Close Price.

    Parameters:
    - actual: DataFrame containing actual Close Price with a DateTime index and a 'Close Price' column.
    - predicted: Series containing forecasted Close Price, with the same index as 'actual'.

    Returns:
    - Displays the plot.
    """

     # Validate inputs
    if not isinstance(actual, pd.DataFrame):
        raise TypeError("Expected 'actual' to be a pandas DataFrame.")
    if not isinstance(predicted, pd.Series):
        raise TypeError("Expected 'predicted' to be a pandas Series with same index (Date) as actual")

    plt.figure(figsize=(10, 5))

    plt.plot(actual.index, actual['Close'], label='Actual Close Price')
    plt.plot(actual.index, predicted, label='Forecasted Close Price', color='red')
    ax = plt.gca()

    ax.spines[['top', 'right']].set_visible(False)
    plt.title('Close Price Forecast vs Actual', fontweight = 'bold', fontsize = '15')
    plt.xlabel('Date')
    plt.ylabel('Close Price')
    plt.legend()
    plt.show()

In [5]:
def inv_diff(last_observed_value, predicted):
    """
    Description:
    Reverts a differenced time series to its original scale

    Parameters:
    - last_observed_value : The last observed value from the original time series.
    - predicted : Series containing the differenced forecasted values to be restored.

    Returns:
    - restored_series : Series with the restored predicted values 

    """    
    current_value = last_observed_value

    inv_diff_values = []

    for diff in predicted:
        inv_diff_value = current_value + diff
        inv_diff_values.append(inv_diff_value)
        current_value = inv_diff_value  

    restored_series = pd.Series(inv_diff_values, index=predicted.index)

    return restored_series

In [6]:
def fcast_evaluation(predicted, actual):
    """
    Description:
    To evaluate forecasting performance using multiple metrics.

    Parameters:
    predicted: Forecasted values.
    actual: Actual observed values.

    Output:
    A dictionary containing the evaluation metrics:
        - 'MSE': Mean Squared Error
        - 'MAE': Mean Absolute Error
        - 'RMSE': Root Mean Squared Error
        - 'MAPE': Mean Absolute Percentage Error
    """

    err= actual - predicted

    # Calculating MSE
    mse = mean_squared_error(actual, predicted)

    # Calculating MAE
    mae = mean_absolute_error(actual, predicted)

    # Calculating RMSE
    rmse = np.sqrt(mse)

    # Calculating MAPE
    abs_percent_err = np.abs(err/actual)
    mape = abs_percent_err.mean() * 100

    return {'MSE': mse,
            'MAE': mae,
            'RMSE': rmse,
            'MAPE': mape
            }

In [7]:
def plt_forecast(predictions, fc_method, train, test):
    """
    Description:
    Plots the training data, validation data (actual), and baseline predictions on a single graph.

    Parameters:
    - predictions : A Series containing the predicted values with date indices.
    - fc_method: A string describing the forecasting method used.
    - train: A dataframe with train data.
    - test: A dataframe with test data

    Output:
    The function creates a plot using Plotly to visualise:
        - Training data
        - Validation data 
        - Baseline forecast predictions

    """
    
    # Plot to visualise the training data, test data and baseline prediction
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=train.index, y=train['Close'], mode='lines', name="Train"))
    fig.add_trace(go.Scatter(x=test.index, y=test['Close'], mode='lines', name="Validation"))
    fig.add_trace(go.Scatter(x=predictions.index, y=predictions, mode='lines', name="Baseline Forecast"))

    fig.update_layout(
        yaxis_title='Close', 
        xaxis_title='Date',
        title= f'{fc_method}'
    )
    fig.show()

## Data Loading
----

ARIMA requires input data to be stationary, therefore I will now be working with the dataset which has the transformed close price.

In [8]:
raw_data = pd.read_csv('../../data/msft_stat.csv', index_col = 0)

In [9]:
# Turn index to DateTime 
raw_data.index = pd.DatetimeIndex(raw_data.index)

In [10]:
raw_data = raw_data.asfreq('D')

### Close BoxCox

In [11]:
adf_test(raw_data['Close_boxcox'])

ADF Statistic: -1.394161651811612
p-value: 0.5850718733487403
Data is not stationary.


Although this data in this column is not stationary, I will be working with this column since differencing is automatically applied in the auto arima model.

## Cross Validation Train/Test Split 
----

In [12]:
tscv = TimeSeriesSplit(n_splits=5)

## Auto-ARIMA Forecasting
-----

In [13]:
from pmdarima import auto_arima


In [14]:
# Setting lam value to lamda value from 03-data-eda
lam = 0.40136288976843615

In [21]:
arima_results_df = pd.DataFrame(columns = ['MSE', 'MAE', 'RMSE', 'MAPE'], index = [1,2,3,4,5])

In [22]:
for i, (train_index, test_index) in enumerate(tscv.split(raw_data)):
    
    # Create train/test datraframes using the indexes from tcsv
    train_df = raw_data.iloc[train_index, :]
    test_df = raw_data.iloc[test_index, :]

    # define y 
    y = train_df['Close_boxcox']

    arima_model = auto_arima(y, seasonal=False, stepwise=True)


    pred_y = arima_model.predict(n_periods=test_df.shape[0])
    pred_y = inv_boxcox(pred_y, lam)
    # Plotting the forecasted values alongside actual data and previous data (training data) 
    print(f"Fold {i+1}:") 
    plt_forecast(pred_y, fc_method='ARIMA', train=train_df, test= test_df)
    print(arima_model.summary())
    
    results_dict = fcast_evaluation(pred_y.values, test_df['Close'].values)
    arima_results_df.iloc[i] = results_dict
    


Fold 1:


                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                  307
Model:               SARIMAX(0, 1, 1)   Log Likelihood                 160.422
Date:                Tue, 17 Sep 2024   AIC                           -314.844
Time:                        11:36:46   BIC                           -303.674
Sample:                    09-19-2019   HQIC                          -310.377
                         - 07-21-2020                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0103      0.007      1.583      0.114      -0.002       0.023
ma.L1         -0.2791      0.028    -10.033      0.000      -0.334      -0.225
sigma2         0.0205      0.001     25.476      0.0

                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                  609
Model:               SARIMAX(1, 1, 0)   Log Likelihood                 377.137
Date:                Tue, 17 Sep 2024   AIC                           -748.274
Time:                        11:36:47   BIC                           -735.044
Sample:                    09-19-2019   HQIC                          -743.127
                         - 05-19-2021                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0089      0.005      1.617      0.106      -0.002       0.020
ar.L1         -0.1895      0.020     -9.352      0.000      -0.229      -0.150
sigma2         0.0169      0.000     36.460      0.0

                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                  911
Model:               SARIMAX(1, 1, 0)   Log Likelihood                 594.183
Date:                Tue, 17 Sep 2024   AIC                          -1182.367
Time:                        11:36:48   BIC                          -1167.927
Sample:                    09-19-2019   HQIC                         -1176.853
                         - 03-17-2022                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0080      0.004      1.862      0.063      -0.000       0.016
ar.L1         -0.1323      0.017     -7.566      0.000      -0.167      -0.098
sigma2         0.0159      0.000     42.301      0.0

                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                 1213
Model:               SARIMAX(2, 1, 2)   Log Likelihood                 706.418
Date:                Tue, 17 Sep 2024   AIC                          -1402.836
Time:                        11:36:52   BIC                          -1377.336
Sample:                    09-19-2019   HQIC                         -1393.235
                         - 01-13-2023                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.1728      0.043      3.994      0.000       0.088       0.258
ar.L2         -0.8819      0.044    -20.083      0.000      -0.968      -0.796
ma.L1         -0.2343      0.041     -5.756      0.0

                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                 1515
Model:               SARIMAX(0, 1, 1)   Log Likelihood                 894.494
Date:                Tue, 17 Sep 2024   AIC                          -1782.987
Time:                        11:36:53   BIC                          -1767.020
Sample:                    09-19-2019   HQIC                         -1777.042
                         - 11-11-2023                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0058      0.003      1.777      0.076      -0.001       0.012
ma.L1         -0.0616      0.016     -3.878      0.000      -0.093      -0.030
sigma2         0.0180      0.000     49.306      0.0

In [25]:
arima_results_df

,MSE,MAE,RMSE,MAPE
1,751.961298,24.810921,27.421913,11.29155
2,961.975342,26.470392,31.015727,8.797296
3,5933.867973,68.230227,77.031604,28.010068
4,5795.963916,68.253401,76.131228,21.35028
5,421.413079,16.670785,20.528348,3.954505


----
**Comment:**

Fold 5 represents the best performance among all folds in terms of evalutaion metrics(see above), so only be looking into model summary for fold 5.

**Model Parameters:**

- p (Autoregressive Term): 0, model does not use past values in the series to forecast future values, main focus of model appears to be recent errors
- d (Differencing Order): 1, one level of differencing needed to make data stationary
- q (Moving Average Term): 1, model uses influence of previous day's error to adjust forecasted values

**Moving Average L1 Coefficient:** -0.0616, indicates a slight overestimate and so the model adjusts predictions downwards slightly to compensate.


## Auto-SARIMA Forecasting
------

In [27]:
sarima_results_df = pd.DataFrame(columns = ['MSE', 'MAE', 'RMSE', 'MAPE'], index = [1,2,3,4,5])

In [31]:
for i, (train_index, test_index) in enumerate(tscv.split(raw_data)):
    
    train_df = raw_data.iloc[train_index, :]
    test_df = raw_data.iloc[test_index, :]

    y = train_df['Close_boxcox']

    sarima_model = auto_arima(y, seasonal=True, m=7, stepwise=True)

    pred_y = sarima_model.predict(n_periods=test_df.shape[0])
    pred_y = inv_boxcox(pred_y, lam)
    
    print(f"Fold {i+1}:") 
    plt_forecast(pred_y, fc_method='SARIMA', train=train_df, test= test_df)
    print(sarima_model.summary())
    
    results_dict = fcast_evaluation(pred_y.values, test_df['Close'].values)
    sarima_results_df.iloc[i] = results_dict


Fold 1:


                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                  307
Model:               SARIMAX(0, 1, 1)   Log Likelihood                 160.422
Date:                Tue, 17 Sep 2024   AIC                           -314.844
Time:                        11:46:00   BIC                           -303.674
Sample:                    09-19-2019   HQIC                          -310.377
                         - 07-21-2020                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0103      0.007      1.583      0.114      -0.002       0.023
ma.L1         -0.2791      0.028    -10.033      0.000      -0.334      -0.225
sigma2         0.0205      0.001     25.476      0.0

                                     SARIMAX Results                                     
Dep. Variable:                                 y   No. Observations:                  609
Model:             SARIMAX(0, 1, 1)x(2, 0, 1, 7)   Log Likelihood                 386.275
Date:                           Tue, 17 Sep 2024   AIC                           -760.550
Time:                                   11:46:18   BIC                           -734.089
Sample:                               09-19-2019   HQIC                          -750.255
                                    - 05-19-2021                                         
Covariance Type:                             opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0023      0.001      2.637      0.008       0.001       0.004
ma.L1         -0.1763      0.022     -8.073

                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                  911
Model:               SARIMAX(1, 1, 0)   Log Likelihood                 594.183
Date:                Tue, 17 Sep 2024   AIC                          -1182.367
Time:                        11:46:20   BIC                          -1167.927
Sample:                    09-19-2019   HQIC                         -1176.853
                         - 03-17-2022                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0080      0.004      1.862      0.063      -0.000       0.016
ar.L1         -0.1323      0.017     -7.566      0.000      -0.167      -0.098
sigma2         0.0159      0.000     42.301      0.0

                                      SARIMAX Results                                      
Dep. Variable:                                   y   No. Observations:                 1213
Model:             SARIMAX(1, 1, 0)x(2, 0, [1], 7)   Log Likelihood                 708.120
Date:                             Tue, 17 Sep 2024   AIC                          -1406.240
Time:                                     11:46:38   BIC                          -1380.740
Sample:                                 09-19-2019   HQIC                         -1396.639
                                      - 01-13-2023                                         
Covariance Type:                               opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.0687      0.018     -3.853      0.000      -0.104      -0.034
ar.S.L7        0.6724      

                                      SARIMAX Results                                      
Dep. Variable:                                   y   No. Observations:                 1515
Model:             SARIMAX(1, 1, 0)x(2, 0, [1], 7)   Log Likelihood                 901.471
Date:                             Tue, 17 Sep 2024   AIC                          -1790.942
Time:                                     11:46:54   BIC                          -1759.007
Sample:                                 09-19-2019   HQIC                         -1779.051
                                      - 11-11-2023                                         
Covariance Type:                               opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0029      0.002      1.656      0.098      -0.001       0.006
ar.L1         -0.0541      

In [30]:
sarima_results_df

,MSE,MAE,RMSE,MAPE
1,751.961298,24.810921,27.421913,11.29155
2,700.305421,22.548885,26.463284,7.500567
3,5933.867973,68.230227,77.031604,28.010068
4,5761.687411,67.969598,75.90578,21.253278
5,553.8382,19.107443,23.533767,4.527409


----
**Comment:**

Again, only looking at model summary for fold 5.


**Non-Seasonal Orders:**

    - p: 1, model using value of 1 previous day to predict the next value
    - d: 1, one level of differencing to make data stationary
    - q: 0, no influence on models previous error

**Seasonal Orders:**

    - P: 2, includes 2 previous seasons (a week) to make forecasts
    - D: 0, no seasonal differencing required, data must already be stationary from non-seasonal differencing
    - Q: 1, model uses error of previous week prediction to adjusts forecasts
    - s: 7, seasonlity of 7 days patterns repeat each week


**Non-Seasonal Coefficients:** 

    - AR.L1 (Autoregressive term): -0.0541, suggests an increase/decrease in price seen in previous day leads to decrease/increase of forecasted value.
    - MA.L1 (Moving Average term):  No moving average terms used for the non-seasonal component as q = 0

**Seasonal Coefficients:** 

    - AR.S.L7: 0.6219, positive coefficient meaning an increase in the price a week a go leads to an increase in forecasted value 7 days later.
    - AR.S.L14: -0.0965, means negative effect from two weeks ago, i.e. increase in price two weeks ago leads to a decreases in forecasted value.
    - MA.S.L7:  -0.5649, indicates an overestimate 7 days ago and so the model adjusts predictions downwards to correct.



## Auto-SARIMAX Forecasting
----

### Adding interest rates

**NOTE:** 

Interest rates affect stock prices in several ways. 

When interest rates are low, it’s cheaper for companies to borrow money, which can lead to more investment and higher profits, often boosting stock prices.

Higher interest rates make bonds and savings accounts more attractive, leading investors to move money away from stocks, which can lower stock prices.

By including interest rates as an exogenous variable, the model can account for these effects use them to improve the accuracy of its predictions.

In [32]:
# Using yfinance to get interest rates
import yfinance as yf

treasury_data = yf.download('^IRX', start=raw_data.index.min(), end=raw_data.index.max())

[*********************100%%**********************]  1 of 1 completed


In [33]:
raw_data.index.max()

Timestamp('2024-09-08 00:00:00')

### Checking for missing dates

In [34]:
first_day = raw_data.index.min()
last_day = raw_data.index.max()

In [35]:
# Calculate difference between last and first day
last_day -  first_day

Timedelta('1816 days 00:00:00')

In [36]:
# Clearly missing dates
treasury_data.shape

(1250, 6)

In [37]:
# Calculate full date range from start to end date
full_index = pd.date_range(start=first_day, end=last_day, freq='D')

In [38]:
# Re-index so there are no more missing dates
treasury_data = treasury_data.reindex(full_index)

In [39]:
full_index.difference(treasury_data.index)

DatetimeIndex([], dtype='datetime64[ns]', freq='D')

In [40]:
treasury_data.isna().sum()

Open         567
High         567
Low          567
Close        567
Adj Close    567
Volume       567
dtype: int64

In [41]:
# Fill in missing data for added dates using interpolation
treasury_data= treasury_data.interpolate(method='linear')

In [42]:
treasury_data.isna().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [43]:
treasury_data = treasury_data.rename(columns={"Adj Close": "interest_rate"})

### Merging with raw_data

In [44]:
# Create new df irt_df to hold raw_data data + interest rate data
# Merging raw_data with interest rate on date index
irt_df = pd.merge(raw_data, treasury_data['interest_rate'], left_index=True, right_index=True)

In [45]:
# Checking for null values 
irt_df.isna().sum()

Open             0
High             0
Low              0
Close            0
Volume           0
Dividends        0
Stock Splits     0
Trend            0
Seasonal         0
Residual         0
Close_boxcox     0
Close_diff       0
interest_rate    0
dtype: int64

In [46]:
irt_df.index

DatetimeIndex(['2019-09-19', '2019-09-20', '2019-09-21', '2019-09-22',
               '2019-09-23', '2019-09-24', '2019-09-25', '2019-09-26',
               '2019-09-27', '2019-09-28',
               ...
               '2024-08-30', '2024-08-31', '2024-09-01', '2024-09-02',
               '2024-09-03', '2024-09-04', '2024-09-05', '2024-09-06',
               '2024-09-07', '2024-09-08'],
              dtype='datetime64[ns]', length=1817, freq='D')

### Adding 5 day moving average

In [47]:
irt_df['5D_M_Avg'] = irt_df['Close_diff'].rolling(window=5).mean()

In [50]:
irt_df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Trend,Seasonal,Residual,Close_boxcox,Close_diff,interest_rate,5D_M_Avg
2019-09-19,134.007496,135.984649,133.787816,134.742966,3.577210e+07,0.0,0.0,132.537949,0.089411,2.115606,15.339153,0.181468,1.883000,NaN
2019-09-20,134.685674,135.296969,132.049466,133.186096,3.916730e+07,0.0,0.0,132.921369,0.217409,0.047318,15.256175,0.108795,1.850000,NaN
2019-09-21,134.118943,134.653829,132.109950,133.090571,3.182463e+07,0.0,0.0,132.920013,0.081202,0.089356,15.251065,0.120711,1.848333,NaN
2019-09-22,133.552212,134.010689,132.170435,132.995046,2.448197e+07,0.0,0.0,133.034633,-0.061176,0.021589,15.245953,0.132649,1.846667,NaN
2019-09-23,132.985481,133.367549,132.230919,132.899521,1.713930e+07,0.0,0.0,132.825862,-0.184360,0.258019,15.240839,0.144610,1.845000,0.137647


### Checking stationarity of X vars

#### Interest Rate

In [51]:
# Check if interest data is stationary 
adf_test(irt_df['interest_rate'])

ADF Statistic: -0.06280367453792138
p-value: 0.9530196964834016
Data is not stationary.


In [472]:
# Apply boxcox to stabalise variance - negative values when using irx so will not do boxcox transform
#irt_df['itr_stat'], _ = boxcox(irt_df['interest_rate'])

In [52]:
# Applying differencing to stabalise the mean 
irt_df['int_r_diff']= irt_df['interest_rate'].diff().dropna()

In [53]:
irt_df = irt_df.dropna()

In [54]:
adf_test(irt_df['int_r_diff'])

ADF Statistic: -6.7900130414130855
p-value: 2.3778534157573166e-09
Data is stationary.


#### 5-Day Moving Average

In [55]:
adf_test(irt_df['5D_M_Avg'])

ADF Statistic: -7.138068335199355
p-value: 3.380090932213932e-10
Data is stationary.


#### Volume

In [56]:
adf_test(irt_df['Volume'])

ADF Statistic: -5.295434776522552
p-value: 5.596149250530376e-06
Data is stationary.


### Auto-SARIMAX

In [60]:
sarimax_results_df = pd.DataFrame(columns = ['MSE', 'MAE', 'RMSE', 'MAPE'], index = [1,2,3,4,5])

In [62]:
raw_data

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Trend,Seasonal,Residual,Close_boxcox,Close_diff
2019-09-19,134.007496,135.984649,133.787816,134.742966,3.577210e+07,0.0,0.0,132.537949,0.089411,2.115606,15.339153,0.181468
2019-09-20,134.685674,135.296969,132.049466,133.186096,3.916730e+07,0.0,0.0,132.921369,0.217409,0.047318,15.256175,0.108795
2019-09-21,134.118943,134.653829,132.109950,133.090571,3.182463e+07,0.0,0.0,132.920013,0.081202,0.089356,15.251065,0.120711
2019-09-22,133.552212,134.010689,132.170435,132.995046,2.448197e+07,0.0,0.0,133.034633,-0.061176,0.021589,15.245953,0.132649
2019-09-23,132.985481,133.367549,132.230919,132.899521,1.713930e+07,0.0,0.0,132.825862,-0.184360,0.258019,15.240839,0.144610
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-04,405.910004,411.239990,404.369995,408.899994,1.513580e+07,0.0,0.0,408.017864,0.133646,0.748485,25.348473,-0.046398
2024-09-05,407.619995,413.100006,406.130005,408.390015,1.419550e+07,0.0,0.0,406.745006,0.089411,1.555598,25.334531,-0.128906
2024-09-06,409.059998,410.649994,400.799988,401.700012,1.960950e+07,0.0,0.0,405.938577,0.217409,-4.455974,25.150673,-0.421629
2024-09-07,408.453328,409.983327,401.249990,403.040009,1.817137e+07,0.0,0.0,406.618578,0.081202,-3.659771,25.187646,-0.332605


In [65]:
for i, (train_index, test_index) in enumerate(tscv.split(irt_df)):
    
    train_df = irt_df.iloc[train_index, :]
    test_df = irt_df.iloc[test_index, :]

    # define X and y
    y = train_df['Close_boxcox']
    X = train_df[['int_r_diff', 'Volume','5D_M_Avg']]

    sarimax_model = auto_arima(y, exogenous = X,seasonal=True, m=7, stepwise=True)
    future_exog = test_df[['int_r_diff', 'Volume', '5D_M_Avg']]

    pred_y = sarimax_model.predict( exog=future_exog, n_periods=test_df.shape[0])
    pred_y = inv_boxcox(pred_y, lam)

    print(f"Fold {i+1}:") 
    plt_forecast(pred_y, fc_method='SARIMAX', train=train_df, test= test_df)
    print(sarimax_model.summary())
    
    results_dict = fcast_evaluation(pred_y.values, test_df['Close'].values)
    sarimax_results_df.iloc[i] = results_dict


Fold 1:


                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                  303
Model:               SARIMAX(0, 1, 1)   Log Likelihood                 156.641
Date:                Tue, 17 Sep 2024   AIC                           -307.282
Time:                        12:39:27   BIC                           -296.151
Sample:                    09-23-2019   HQIC                          -302.829
                         - 07-21-2020                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0108      0.007      1.630      0.103      -0.002       0.024
ma.L1         -0.2802      0.028     -9.957      0.000      -0.335      -0.225
sigma2         0.0207      0.001     25.178      0.0

                                     SARIMAX Results                                     
Dep. Variable:                                 y   No. Observations:                  605
Model:             SARIMAX(0, 1, 1)x(2, 0, 1, 7)   Log Likelihood                 381.984
Date:                           Tue, 17 Sep 2024   AIC                           -751.968
Time:                                   12:39:44   BIC                           -725.547
Sample:                               09-23-2019   HQIC                          -741.686
                                    - 05-19-2021                                         
Covariance Type:                             opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0023      0.001      2.653      0.008       0.001       0.004
ma.L1         -0.1773      0.022     -8.076

                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                  907
Model:               SARIMAX(1, 1, 0)   Log Likelihood                 589.872
Date:                Tue, 17 Sep 2024   AIC                          -1173.743
Time:                        12:39:46   BIC                          -1159.316
Sample:                    09-23-2019   HQIC                         -1168.234
                         - 03-17-2022                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0081      0.004      1.891      0.059      -0.000       0.017
ar.L1         -0.1326      0.018     -7.552      0.000      -0.167      -0.098
sigma2         0.0159      0.000     42.135      0.0

                                      SARIMAX Results                                      
Dep. Variable:                                   y   No. Observations:                 1209
Model:             SARIMAX(1, 1, 0)x(2, 0, [1], 7)   Log Likelihood                 703.930
Date:                             Tue, 17 Sep 2024   AIC                          -1397.860
Time:                                     12:40:04   BIC                          -1372.377
Sample:                                 09-23-2019   HQIC                         -1388.264
                                      - 01-13-2023                                         
Covariance Type:                               opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.0688      0.018     -3.846      0.000      -0.104      -0.034
ar.S.L7        0.6720      

                                     SARIMAX Results                                     
Dep. Variable:                                 y   No. Observations:                 1511
Model:             SARIMAX(0, 1, 1)x(2, 0, 1, 7)   Log Likelihood                 897.329
Date:                           Tue, 17 Sep 2024   AIC                          -1782.659
Time:                                   12:40:17   BIC                          -1750.739
Sample:                               09-23-2019   HQIC                         -1770.772
                                    - 11-11-2023                                         
Covariance Type:                             opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0028      0.002      1.663      0.096      -0.000       0.006
ma.L1         -0.0554      0.016     -3.387

In [66]:
sarimax_results_df

,MSE,MAE,RMSE,MAPE
1,870.003021,26.650528,29.495814,12.107221
2,682.694299,22.297619,26.128419,7.422157
3,6055.936195,68.911981,77.819896,28.289768
4,5762.285077,67.973902,75.909717,21.254699
5,542.694046,18.925814,23.295795,4.485299


----
**Comment:**

Again, only looking at model summary for fold 5 as this fold shows the best performance.


**Non-Seasonal Orders:**

    - p: 1, no autoregressive terms used in the model
    - d: 1, one level of differencing to make data stationary
    - q: 0, model uses influence of previous day's error to adjust forecasted values

**Seasonal Orders:**

    - P: 2, includes 2 previous seasons (a week) to make forecasts
    - D: 0, no seasonal differencing required, data must already be stationary from non-seasonal differencing
    - Q: 1, model uses error of previous week prediction to adjusts forecasts
    - s: 7, seasonlity of 7 days patterns repeat each week

**Non-Seasonal Coefficients:** 

    - MA.L1 (Moving Average term): -0.0554, indicates a slight adjustment downward in forecasts due to errors from the previous day.

**Seasonal Coefficients:** 

    - AR.S.L7: 0.6218 – A positive coefficient suggesting that an increase in the price from one week ago leads to a higher forecasted value.
    - AR.S.L14: -0.0961 – A negative coefficient indicating that an increase in price two weeks ago results in a decrease in the forecasted value.
    - MA.S.L7: -0.5653 – A negative coefficient shows that if there was an overestimate 7 days ago, the model adjusts the forecast downward to correct for this.

**Exogenous Variables:** 

Since I am using Auto-SARIMAX, I am unable to see the X variables I added in the model summary.

To inspect these variables, I will have to manually run SARIMAX with the above parameters and take a look.

### Manual SARIMAX

In [68]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [75]:
# Using train_df, test_df (still set to fold 5 dates) and X and y defined in loop
sarimax_model = SARIMAX(y, exog=X, order=(0, 1, 1), seasonal_order=(2, 0, 1, 7))
result = sarimax_model.fit()

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            8     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -6.51472D-01    |proj g|=  1.74528D+10


 This problem is unconstrained.



           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    8      1     21      1     0     0   1.745D+10  -6.515D-01
  F = -0.65147236820364984     

ABNORMAL_TERMINATION_IN_LNSRCH                              


/Users/simybasra/anaconda3/envs/timeseries/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals


 Line search cannot locate an adequate point after MAXLS
  function and gradient evaluations.
  Previous x, f and g restored.
 Possible causes: 1 error in function or gradient evaluation;
                  2 rounding error dominate computation.


In [76]:
result.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                                     SARIMAX Results                                     
=========================================================================================
Dep. Variable:                      Close_boxcox   No. Observations:                 1511
Model:             SARIMAX(0, 1, 1)x(2, 0, 1, 7)   Log Likelihood                 984.375
Date:                           Tue, 17 Sep 2024   AIC                          -1952.749
Time:                                   12:53:37   BIC                          -1910.191
Sample:                               09-23-2019   HQIC                         -1936.900
                                    - 11-11-2023                                         
Covariance Type:                             opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
int_r_diff    -0.1824   2.27e-07  -8.05e+05      0.000      -0.182      -0.182
Volume     -1.589e-09   2.32e-10     -6.841      0.000   -2.04e-09   -1.13e-09
5D_M_Avg       0.5153   6.09e-07   8.46e+05      0.000       0.515       0.515
ma.L1         -0.0912   2.51e-06  -3.64e+04      0.000      -0.091      -0.091
ar.S.L7       -0.2433   1.14e-06  -2.13e+05      0.000      -0.243      -0.243
ar.S.L14      -0.0381   1.57e-06  -2.43e+04      0.000      -0.038      -0.038
ma.S.L7        0.3901   1.51e-06   2.58e+05      0.000       0.390       0.390
sigma2         0.0161      0.000     52.740      0.000       0.015       0.017
===================================================================================
Ljung-Box (L1) (Q):                   0.03   Jarque-Bera (JB):              1924.52
Prob(Q):                              0.86   Prob(JB):                         0.00
Heteroskedasticity (H):               1.09   Skew:                             0.39
Prob(H) (two-sided):                  0.33   Kurtosis:                         8.47
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
[2] Covariance matrix is singular or near-singular, with condition number 3.07e+21. Standard errors may be unstable.
"""

----
**Comment:**

**Exogenous Variables Coefficients:**

    - interest rate: -0.1824, showing a strong negative effect on the forecast. Higher interest rates are associated with lower forecasted values, this makes sense since increased interest rates generally slow market activity and potentially lower stock prices.

    - Volume: -1.589e-09, a very small negative effect on the forecast, volumne has hardly any effect on the forecasted value. This suggests trading volumne does not have a strong impact on the stock price.

    - 5 Day Moving Avg: 0.5153, shows a strong positive effect on the forecast. A higher 5 day average tends to lead to a substantial increase in the forecasted value, this makes sense as a higher rolling average over the past five days indicates an upward trend in prices, which would lead to higher forecasted values.

## Conclusion
----

In this notebook, I have explored the three advanced forecasting mehtods (ARIMA, SARIMA and SARIMAX). Despite an advancement in model complexity, the results show little improvements. This highlights the challenges faced in predicitng stock prices. Stocks are highly volatile and are heavily influenced by events which are unpredictable.

The volatility observed in the stock data is largely due to occurrence of external events which are impossible to predict such as COVID-19. Unpredictable events led to sharp fluctuations in prices, which made it very difficult to forecast stock prices. 

Given what I have found so far, in the future I would like to expand this project by:

1. Feature Engineering

    - I performed feature engineering to a degree when thinking of some exogenous variables to include for SARIMAX forecasting. To take this further it may be better to predict a feature which is not Close price. For example, prediciting whether or not a stockholder would see a return on their investment may be a simpler project and make forecasting more manageable. 

2. Advanved Models

    - Explore even more complex methods to create the predictions such as XGBoost or LSTMs 
    - These models may provide a better forecast but I still think the improvement will be marginal in comparison to the increase in model complexity unless I was to change what I predict.

Overall, while the current models offer valuable insights, I would like to revisit this project again in the future and take into account some of the above points.